In [1]:
import os
import numpy as np
import json

from PIL import Image  
from csv import writer
import csv

from tensorflow import keras
import tensorflow as tf

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jun  6 12:48:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
WIDTH = 730
HEIGHT = 530
CHANNELS = 3

BATCH_SIZE = 8
TRAIN_DATASET_NO = 3136

TEST_DATASET_NO = 348

SAMPLES_NO = TRAIN_DATASET_NO // BATCH_SIZE

DATA_PATH = "datasets/SUNRGB-D3D/SUNRGBD/kv2/kinect2data"
CSV_EXISTS = True


X_TRAIN_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/train_images_k2.csv'
y_TRAIN_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/train_annotations_k2.csv'
X_TEST_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/test_images_k2.csv'
y_TEST_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/test_annotations_k2.csv'
X_PREDICT_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/predict_k2.csv'

In [4]:
def csv_lazy(file_name):
    with open(file_name) as f:
        r = csv.reader(f)
        
        for row in r:
            yield row

def batch_generator(X_file, batch_size):

    csv_generator = csv_lazy(X_file)

    while True:
        batch = np.array([])

        for i in range(batch_size):
            batch = np.append(batch, np.array(next(csv_generator)).astype('float') / 255.0)
        
        yield batch.reshape(batch_size, WIDTH, HEIGHT, CHANNELS).astype('float')

def label_generator(y_file, batch_size):
    csv_generator = csv_lazy(y_file)

    while True:
        batch = np.array([])

        for i in range(batch_size):
            batch = np.append(batch, next(csv_generator))
        
        yield batch.reshape(batch_size, 10).astype('float')
        
def model_generator(X_file, y_file, batch_size, limit_iteration):
    X_generator = batch_generator(X_file, batch_size)
    y_generator = label_generator(y_file, batch_size)
    
    iteration_no = 0

    while True:
        
        iteration_no += 1

        print('Iteration: ' + str(iteration_no))

        # Reset to start of file
        if iteration_no > limit_iteration:
            X_generator = batch_generator(X_file, batch_size)
            y_generator = label_generator(y_file, batch_size)
            iteration_no = 0

        X_batch = next(X_generator)
        y_batch = next(y_generator)

        yield (X_batch, y_batch)

In [5]:
base_model = keras.applications.ResNet50(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(10)(avg)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

model = keras.models.Model(inputs=base_model.input, outputs=loc_output)
model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

94781440/94765736 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [6]:
model.fit(x = model_generator(X_TRAIN_DATA_PATH, y_TRAIN_DATA_PATH, BATCH_SIZE, SAMPLES_NO // 10),
          steps_per_epoch = SAMPLES_NO // 10,
          epochs = 10,
          verbose = 2
          )

Iteration: 1
Epoch 1/10
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
39/39 - 198s - loss: 1.0676 - accuracy: 0.4199 - 198s/epoch - 5s/step
Epoch 2/10
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
It

In [7]:
model.evaluate(
    x = model_generator(X_TEST_DATA_PATH, y_TEST_DATA_PATH, 4, TEST_DATASET_NO // 4),
    verbose = 2,
    steps = TEST_DATASET_NO // 4 - 1
)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

[5.587038040161133, 0.3517441749572754]

In [8]:
base_model = keras.applications.ResNet50(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(10)(avg)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

model = keras.models.Model(inputs=base_model.input, outputs=loc_output)
model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [9]:
model.fit(x = model_generator(X_TRAIN_DATA_PATH, y_TRAIN_DATA_PATH, BATCH_SIZE, SAMPLES_NO // 4),
          steps_per_epoch = SAMPLES_NO // 4,
          epochs = 10,
          verbose = 2
          )

Iteration: 1
Epoch 1/10
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iter

In [10]:
model.evaluate(
    x = model_generator(X_TEST_DATA_PATH, y_TEST_DATA_PATH, 4, TEST_DATASET_NO // 4),
    verbose = 2,
    steps = TEST_DATASET_NO // 4 - 1
)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

[2.2031850814819336, 0.1744185984134674]

In [11]:
model.predict(
    x = batch_generator(X_PREDICT_DATA_PATH, 10),
    verbose = 2,
    steps = 1
)

1/1 - 3s - 3s/epoch - 3s/step


array([[ 9.0707399e-02,  1.4420017e+00, -1.1399061e-01, -1.3524091e-01,
        -1.6034013e-01,  3.5219279e-01,  3.0778117e+00,  3.3338571e+00,
         3.8890080e+00,  3.3730831e+00],
       [ 3.0474776e-01,  1.5717343e+00, -4.9745223e-01, -5.7251418e-01,
        -9.5069343e-01, -9.2466813e-01,  2.6191871e+00,  3.0240769e+00,
         3.2679114e+00,  2.7774987e+00],
       [ 2.6931241e-01,  1.3792093e+00, -5.6806648e-01, -1.0652882e-01,
        -5.1476663e-01, -7.6528466e-01,  2.3265488e+00,  2.8996499e+00,
         3.6338711e+00,  2.9804928e+00],
       [-9.0315267e-03,  1.0868754e+00,  9.7779769e-01,  7.9214317e-01,
         8.2349664e-01,  2.2323611e+00,  3.8769152e+00,  4.1456170e+00,
         4.3106780e+00,  4.2891092e+00],
       [-1.4975467e-01,  1.2054889e+00,  5.4675829e-01, -5.6591737e-01,
        -6.1661553e-01,  1.2484546e+00,  3.1191709e+00,  3.3515158e+00,
         3.2685862e+00,  2.7208617e+00],
       [ 3.8368374e-02,  1.3780172e+00,  1.7824064e-01, -9.2989087e-01,
   

In [12]:
model.save('/content/drive/MyDrive/Colab Datasets/ResNet50')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Datasets/ResNet50/assets
